# fMRI analysis and visualization - subject averaged data from the motor task.  

#### In this notebook we will do some visualization and analysis of fMRI data which has already been averaged across samples in each participant.

#### For this example, I chose the Motor Task from the hcp_task data set.  

#### The motor task is a good place to start, because we can use a simple fact to guide us.  Movement of the left side of the body involves activity of the right motor cortex, while movement of the right side of the body involves activity of the left motor cortex.  

#### In this example, I have preprocessed the data by averaging together the scans of each experimental condition (type of movement) for each subject, and then provide the data for 100 subjects in one file.

#### When each subject's data was averaged by condition, I first removed the mean across all the scans - each subject's data averages to zero.  

#### (I made a mistake when creating this file by not saving the fact that each subject average in each condition is based on 64 scans)

#### load the the modules needed

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt 
from hdf5storage import loadmat, savemat 
from nilearn import plotting,datasets

In [ ]:
#IF YOU MOVED THE PROGRAMS INTO THE TOP LEVEL OF hcp_task
#hcppath = './'
#datapath = './processed'
#or else write the full path
hcppath = '/home/ramesh/Teaching/classdata/fmri/hcp_task/'
datapath = hcppath+'processed/'


#### Load the region labels file, regions.npy

In [ ]:
regions = np.load(hcppath+'regions.npy') # this is the file 
roi_names = regions[:,0] # these are the names of each of 360 roi from the parcellation.
network_names = regions[:,1] # these are the networks each roi "belongs" to
networks = np.unique(regions[:,1]) # these are the unique network names 

#### Load the hcp atlas file. 

In [ ]:
atlas = dict(np.load(hcppath+'hcp_atlas.npz'))
fsaverage = datasets.fetch_surf_fsaverage()

#### Load the fMRI data

In [ ]:
data = loadmat(datapath+'MOTOR_fmri_subjectaverage.mat')

In [ ]:
data.keys()

#### Extract the data in the dictionary into separate variables

In [ ]:
condition_index = data['condition_index']
conditions = data['conditions']
fmri = data['fmri']
nconditions = data['nconditions']
nregions = data['nregions']
nsubjects = data['nsubjects']
subject = data['subject']
task = data['task']

## README 

#### condition_index - index for each data sample, indicating the experimental condition 
#### conditions - conditions in the experiment 
#### fmri - fmri data averaged over participants, nregions x (nsubjects x nconditions)
#### nconditions - number of conditions
#### nregions - number of regions (always 360)
#### nsubjects - number of subjects (always 100)
#### subject - indexes which subject each average comes from. 
#### task - which task the data comes from.  

In [ ]:
print(conditions)

#### For the MOTOR task 
* #### 'lf' - left foot, condition_index = 0 
* #### 'rf' - right foot, condition_index = 1
* #### 'lh' - left hand, condition_index = 2 
* #### 'rh' - right hand, condition_index = 3
* #### 't' - tongue, condition_index = 4 
* #### 'cue' - when the visual cue was presented (no movement), condition_index = 5



#### 1.  First I'm going to compute some averages by condition. 

In [ ]:
avg_fmri = dict()
#lh 
avg_fmri['lh'] = np.mean(fmri[:,condition_index == 0],axis = 1)
avg_fmri['rh'] = np.mean(fmri[:,condition_index == 1],axis = 1)
avg_fmri['lf'] = np.mean(fmri[:,condition_index == 2],axis = 1)
avg_fmri['rf'] = np.mean(fmri[:,condition_index == 3],axis = 1)
avg_fmri['t'] = np.mean(fmri[:,condition_index == 4],axis = 1)
avg_fmri['cue'] = np.mean(fmri[:,condition_index == 5],axis = 1)


2. Visualize the data on the brain.  

#### To do this, I am going to directly map the 

`plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = 'lf',black_bg = True)`

#### Note that in this version, I chose symmetric_cmap = True.  This is a good default choice.  


In [ ]:
plot_condition = 'rf' #select the condition I want to plot 
surf_label = avg_fmri[plot_condition][atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_right'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

In [ ]:
surf_label = avg_fmri[plot_condition][atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True,title = plot_condition,black_bg = True)

### 2. Let's plot the difference between conditions. Before I do that, I am going to z-score the data

#### The variables in the experiment are the 360 ROI.   I must make the mean of these variables zero in order to center the data on zero.  

In [ ]:
from scipy.stats import zscore
z = zscore(fmri)

In [ ]:
#To comparse left foor to right foot, I see that condition_index is 0, and condition_index is 1.  
diff = np.mean((z[:,condition_index == 0]-z[:,condition_index == 1]),axis = 1)
plt.figure(figsize = (8,8))
#plt.plot(diff)
plt.bar(np.arange(180),diff[0:180],color = 'r')
plt.bar(np.arange(180,360,1),diff[180:360],color = 'b')
plt.xticks(range(0,390,30))
plt.legend(('Right Hemisphere','Left Hemisphere'))
plt.xlabel('ROI')
plt.ylabel('fmri difference (std-units)')
plt.grid()

### 3. Lets identify the region with the strongest positive and negative difference  

In [ ]:
r1= np.argmax(diff)
print(roi_names[r1],network_names[r1])
r2 = np.argmin(diff)
print(roi_names[r2],network_names[r2])

#### Interestingly - Neither of these regions is the primary motor cortex (Area 4)
#### Area 5 is part of the superior parietal lobule, and is right next to somatosensory cortex


In [ ]:
surf_label = diff[atlas['labels_R']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['pial_right'],surf_label,symmetric_cmap = True,title = 'lf-rf',black_bg = True)

In [ ]:
surf_label = diff[atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['pial_left'],surf_label,symmetric_cmap = True,title = 'lf-rf',black_bg = True)

#### Here I identify the 10 regions with largest differences. 

In [ ]:
n = 10
ordered_index = np.argsort(np.abs(diff)) # sorts into ascending order 
topn = ordered_index[-n:] #take the last n
for j in range(len(topn)):
    print(roi_names[topn[j]],network_names[topn[j]])


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
z_T = np.transpose(z)
from sklearn.model_selection import KFold 
kf = KFold(n_splits = 5,shuffle = True)
errors = 0
predictions = np.zeros(np.shape(z_T)[0])
probability = np.zeros((len(predictions),nconditions))
for train_index, test_index in kf.split(z_T):
#    print("TRAIN:", train_index, "TEST:", test_index)
    data_train = z_T[train_index]
    data_test = z_T[test_index]
    Label_train= condition_index[train_index] 
    Label_test = condition_index[test_index]
    lda = LDA(n_components=1)
    data_model = lda.fit(data_train, Label_train)
    predict = lda.predict(data_test)
    test = Label_test == predict
    errors = errors + sum(~test)
    predictions[test_index] = predict
    probability[test_index,:] = lda.predict_proba(data_test)
errorrate = errors/len(condition_index)
print(errorrate)

#### Lets compute a confusion matrix

In [ ]:
confusion_matrix = np.zeros((nconditions,nconditions))
for j in range(nconditions):
    values, counts = np.unique(predictions[condition_index == j],return_counts = True)
    confusion_matrix[j,values.astype(int)] = counts
print(confusion_matrix)    

In [ ]:
#make a nice plot of the confusion matrix
plt.figure(figsize = (8,8))
plt.imshow(confusion_matrix,vmin = 0, vmax = 10, cmap = 'jet')
plt.xticks(range(6),labels = conditions)
plt.yticks(range(6),labels = conditions)
plt.xlabel('Predicted Condition')
plt.ylabel('Actual Condition')
plt.colorbar()

#### LDA Analysis.  

In [ ]:
fmri_lda = LDA(n_components=1)
fmri_model = fmri_lda.fit(z_T, condition_index)
fmri_predict = fmri_lda.transform(z_T)
fmri_coef = fmri_lda.coef_

#### make a map of the weights.  

In [ ]:
ci = 4 # index for the condition
surf_label = fmri_coef[ci,atlas['labels_L']] ## This maps the 360 values onto every voxel on the ROI in the brain.  
plotting.view_surf(fsaverage['infl_left'],surf_label,symmetric_cmap = True, title = conditions[ci],black_bg = True, vmax = 20)

#### Lets visualize the distribution of outputs of each projection. 

In [ ]:
lda_data = fmri_coef@z; #(There is a built in method decision_variable that does this)
for j in range(nconditions):
    plt.figure(figsize = (6,6))
    plt.hist(lda_data[j,condition_index==j])
    plt.hist(lda_data[j,condition_index!=j])
    plt.legend((conditions[j],'All others'))
    plt.show()

#plt.hist(test[0,:][condition_index!=0],bins = np.arange(-20,20,1))
plt.show()